In [2]:
import os
import sys               
sys.path.append(os.path.dirname(os.path.abspath("."))) 
from API_KEYS import InitAzureEnv,AzureDeploymentName

#初始化Azure环境
InitAzureEnv()

In [3]:
from langchain.chat_models import AzureChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [4]:
chat = AzureChatOpenAI(
    deployment_name=AzureDeploymentName.CHAT,
    temperature=0
)

In [52]:
system_temp='''你是一个SQL开发人员,需要根据客户的输入,分析出他查询的是什么产品,什么表的什么字段?用json{{'system':'','table':'','col':''}}的格式告诉我。只要json，不需要其他信息'''

system_msg_prompt = SystemMessagePromptTemplate.from_template(system_temp)
human_msg_prompt = HumanMessagePromptTemplate.from_template("{text}")

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])
ai_message = chat(chat_prompt.format_prompt(text="从售楼系统的房间表中获取2023年1季度的最低折扣").to_messages())

In [53]:
ai_message.content.replace("\n", "").replace(" ", "")

"{'system':'售楼系统','table':'房间宽表','col':'最低折扣','year':'2023','quarter':'1'}"

In [43]:
import pymysql
conn = pymysql.connect(
    host="localhost", 
    port=3306,
    user="root", 
    password="123456", 
    db="mdc"
)

In [44]:
cursor = conn.cursor()

In [58]:
cursor.execute('''SELECT ApplicationName from mdc_app''')
systems = cursor.fetchall()

In [63]:
for sys in systems:
    print(sys[0])

风控中心
流程中心-引擎
动态货值
成本系统
招采系统
流程中心-业务
移动验房
云物业
其他云链
云链
销售系统
材料供应链
数据服务中心
主数据
投资收益
费用系统
工作流
其它ERP子系统
明源新一代云ERP
全面预算
第三方应用
第三方应用
佣金管理
云租赁
云物业(云空间)
云资管-CDM
云资管-DM
云资管-标准DM
其他云空间
云空间
投后管理
在线投标
其它云产品
云客
云助手
移动报表
移动审批
移动计划
云助手
全面预算
费用系统
采招系统
成本系统
计划系统
销售系统
工作流
其它ERP子系统
明源核心ERP
计划系统


In [64]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import json
from langchain.docstore.document import Document

embeddings = OpenAIEmbeddings(
    deployment=AzureDeploymentName.EMEDDINGS,
    chunk_size=1,
)
chroma_systems_directory = os.path.abspath("chroma/systems")

systems_pages = []
for sys in systems:
    systems_pages.append(Document(page_content=sys[0],metadata=dict()))

docsearch = Chroma.from_documents(systems_pages, embeddings, persist_directory=chroma_systems_directory)
docsearch.persist()

Using embedded DuckDB with persistence: data will be stored in: /Users/jiangyy/Codes/AI/LangChainCn/03-Samples/chroma/systems


In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate

# # 加载数据
sys_search = Chroma(persist_directory=chroma_systems_directory, embedding_function=embeddings)

system_table_similarity = sys_search.similarity_search(query="售楼系统",k=1)

In [89]:
print(system_table_similarity[0].page_content)

销售系统


In [56]:
cursor.execute('''SELECT displayName from mdc_table''')
tables = cursor.fetchall()

In [67]:
chroma_tables_directory = os.path.abspath("chroma/tables")

tables_pages = []
for tab in tables:
    tables_pages.append(Document(page_content=tab[0],metadata=dict()))

tab_search = Chroma.from_documents(tables_pages, embeddings, persist_directory=chroma_tables_directory)
tab_search.persist()

Using embedded DuckDB with persistence: data will be stored in: /Users/jiangyy/Codes/AI/LangChainCn/03-Samples/chroma/tables


In [ ]:
tab_search = Chroma(persist_directory=chroma_tables_directory, embedding_function=embeddings)

table_similarity = tab_search.similarity_search(query="房间表",k=1)

In [90]:
print(table_similarity[0].page_content)

房间宽表


In [91]:
cursor.execute('''SELECT
        mdc_table.TableName
    FROM mdc_table
    INNER JOIN mdc_app ON mdc_table.appcode = mdc_app.appcode and mdc_table.AppLevelCode = mdc_app.levelcode
    where mdc_app.ApplicationName='销售系统' and mdc_table.DisplayName = '房间宽表'
    ORDER BY  TableName''')
tablecode = cursor.fetchone()

In [93]:
print(tablecode[0])

data_wide_s_Room


In [94]:
system_temp='''你是一个SQL开发人员,需要根据客户的输入生成一段SQL语句, 其中`售楼系统`的`房间表`对应的数据库表是data_wide_s_Room'''
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_temp)

human_msg_prompt = HumanMessagePromptTemplate.from_template("{text}")

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])
ai_message = chat(chat_prompt.format_prompt(text="从售楼系统的房间表中获取2023年1季度的最低折扣").to_messages())

In [95]:
ai_message

AIMessage(content='生成的SQL语句如下：\n\n```sql\nSELECT MIN(discount) FROM data_wide_s_Room WHERE YEAR(sale_date) = 2023 AND QUARTER(sale_date) = 1;\n```\n\n这条SQL语句使用了`YEAR()`和`QUARTER()`函数来获取`sale_date`字段的年份和季度，然后使用`MIN()`函数获取最低折扣。', additional_kwargs={}, example=False)